In [1]:
from utils_classiff import create_datasets_and_directories,BeeBatchGenerator,create_directories,load_img
import os 

2023-05-26 16:26:58.511884: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-26 16:26:58.646187: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-26 16:26:59.339656: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/basile/.local/lib/python3.10/site-packages/cv2/../../lib64:
2023-05-26 16:26:59.339763: W te

In [5]:
##### PARAMETERS #####
PATH_TO_DATASET ="/home/basile/Documents/projet_bees_detection_basile/bees_detection/src/datafiles/final_datafiles/dataset_yolo_cropped_with_cleaned_structure.csv"
OUTPUT_FOLDER = "/home/basile/Documents/projet_bees_detection_basile/data_bees_detection/benchmark_classification/2_Resnet_300_imgs_2_species/"
IMG_SIZE = 256
CAP = 20000
NB_IMG_TO_KEEP = 300
NB_CLASSES = 2    
BACTH_SIZE = 16
##### PARAMETERS #####

# Create the datasets and directories
X_train, y_train, X_val, y_val, X_test, y_test, classes = create_directories(path_to_csv=PATH_TO_DATASET, path_to_output=OUTPUT_FOLDER, image_size=IMG_SIZE, cap=CAP, nb_img_to_keep=NB_IMG_TO_KEEP,nb_classes_to_keep=NB_CLASSES)

list = ['train', 'test', 'validation']
NB_CLASSES = len(classes)


paths = [os.path.join(OUTPUT_FOLDER, i) for i in list]

x_train, y_train = load_img(paths[0],img_size = IMG_SIZE,classes = classes)
x_test, y_test = load_img(paths[1],img_size = IMG_SIZE,classes = classes)
x_val, y_val = load_img(paths[2],img_size = IMG_SIZE,classes = classes)

# Normalize the data
x_train = x_train/255
x_val = x_val/255
x_test = x_test/255

nb_classes_to_keep must be less than the number of species with more than 20000 images, we kept all the species

 SUM UP OF THE FILTERING PROCESS : 

--------------------------------------------------
Number of species with more than 20000 images : 2
Number of species kept after filtering : 2
__________________________________________________
Number of species in the train dataset : 2
Number of images in the train dataset : 420
--------------------------------------------------
Number of species in the validation dataset : 2
Number of images in the validation dataset : 90
--------------------------------------------------
Number of species in the test dataset : 2
Number of images in the test dataset : 90
--------------------------------------------------
Classes : 
['Apis mellifera', 'Bombus terrestris']
__________________________________________________


Make simplest Resnet model

In [9]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# conv base
conv_base = ResNet50(weights=None,input_shape=(IMG_SIZE,IMG_SIZE,3),include_top=False)

# classifier
x = conv_base.output
x = Flatten()(x)
x = Dense(512,activation='relu')(x)
predictions = Dense(NB_CLASSES,activation='softmax')(x)

# add classifier to conv base
model = Model(inputs=conv_base.input,outputs=predictions)

opti = Adam(learning_rate=0.0001)

# Compile the model
model.compile(optimizer=opti,loss='categorical_crossentropy',metrics=['accuracy'])


In [10]:
history = model.fit(x_train,y_train,epochs=10,validation_data=(x_val,y_val),batch_size=BACTH_SIZE)

Epoch 1/10


2023-05-26 16:33:52.076600: W tensorflow/tsl/framework/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 256.00MiB (rounded to 268435456)requested by op Fill
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-05-26 16:33:52.076659: I tensorflow/tsl/framework/bfc_allocator.cc:1034] BFCAllocator dump for GPU_0_bfc
2023-05-26 16:33:52.076664: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (256): 	Total Chunks: 208, Chunks in use: 208. 52.0KiB allocated for chunks. 52.0KiB in use in bin. 38.8KiB client-requested in use in bin.
2023-05-26 16:33:52.076667: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (512): 	Total Chunks: 176, Chunks in use: 176. 88.0KiB allocated for chunks. 88.0KiB in use in bin. 88.0KiB client-requested in use in bin.
2023-05-26 16:33:52.076671: I tensorflow/tsl/framew

ResourceExhaustedError: in user code:

    File "/home/basile/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/home/basile/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/basile/.local/lib/python3.10/site-packages/tensorflow/python/distribute/distribute_lib.py", line 1316, in run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    File "/home/basile/.local/lib/python3.10/site-packages/tensorflow/python/distribute/distribute_lib.py", line 2895, in call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    File "/home/basile/.local/lib/python3.10/site-packages/tensorflow/python/distribute/distribute_lib.py", line 3696, in _call_for_each_replica
        return fn(*args, **kwargs)
    File "/home